In [11]:
# ============================================================
# PRODUCTION PLANNING MILP – 3 DRINKS, 3 MACHINES, 3 SCENARIOS
# ============================================================

import pulp as pl
import pandas as pd

# ------------------------------------------------------------
# 1. SETS AND CORE PARAMETERS
# ------------------------------------------------------------

# Products
# P1 = Ready-to-drink protein shake
# P2 = Zero-sugar energy drink
# P3 = Functional flavored water (electrolytes)
products = ["P1", "P2", "P3"]

# Machines
# M1 = Mixing / blending
# M2 = Filling & capping
# M3 = Packing / case packing
machines = ["M1", "M2", "M3"]

# Planning horizon (7 days)
days = list(range(1, 8))
T = days[-1]

# ---------- Processing times (hours per unit) ----------
# These are realistic: ~1.2–1.8 minutes on Mix, smaller on Fill/Pack.

proc_time = {
    ("P1", "M1"): 0.030,  ("P1", "M2"): 0.010, ("P1", "M3"): 0.008,
    ("P2", "M1"): 0.025,  ("P2", "M2"): 0.009, ("P2", "M3"): 0.007,
    ("P3", "M1"): 0.020,  ("P3", "M2"): 0.008, ("P3", "M3"): 0.006,
}

# ---------- Setup times (hours per product produced on that day) ----------
# One setup per product per machine per day.
setup_time = {
    "M1": 2.0,
    "M2": 1.5,
    "M3": 1.0,
}

# ---------- Big-M upper bounds for daily production ----------
# Should be higher than any realistic daily volume.
M_big = {
    "P1": 1500,
    "P2": 1500,
    "P3": 1500,
}

# ---------- Cost parameters ----------
# Production & inventory are cheap compared to lateness and setups.
prod_cost = {"P1": 1.0,  "P2": 1.0,  "P3": 1.0}
inv_cost  = {"P1": 0.05, "P2": 0.05, "P3": 0.05}

# Late penalties (per unit short at end of week)
# P1 is most critical (flagship protein shake)
late_penalty = {"P1": 15.0, "P2": 10.0, "P3": 8.0}


# ------------------------------------------------------------
# 2. HELPER: BUILD AND SOLVE ONE SCENARIO
# ------------------------------------------------------------

def solve_scenario(
    scenario_name: str,
    hours_per_day: dict,      # {machine: hours per day}
    demand_week: dict,        # {product: weekly demand}
    setup_cost_value: float   # cost per product-setup per day
):
    """
    Build and solve a mixed-integer linear model for one scenario.
    Returns schedule, utilization, avg utilization, late, throughput, KPI dict.
    """

    # ----- Model -----
    model = pl.LpProblem(f"ProdPlan_{scenario_name}", pl.LpMinimize)

    # ----- Decision variables -----
    # x[p,t]   = quantity produced
    # I[p,t]   = inventory end of day
    # y[p,t]   = 1 if product p is produced that day (setup)
    # late[p]  = unmet demand at end of horizon

    x = pl.LpVariable.dicts("x", (products, days), lowBound=0, cat="Continuous")
    I = pl.LpVariable.dicts("I", (products, days), lowBound=0, cat="Continuous")
    y = pl.LpVariable.dicts("y", (products, days), lowBound=0, upBound=1, cat="Binary")
    late = pl.LpVariable.dicts("late", products, lowBound=0, cat="Continuous")

    # ----- Inventory balance & demand satisfaction -----
    # Start with zero inventory; only end-of-week demand matters.

    for p in products:
        for t in days:
            if t == days[0]:
                # Day 1: I[p,1] = x[p,1]
                model += I[p][t] == x[p][t], f"InvBal_{p}_{t}"
            else:
                # I[p,t] = I[p,t-1] + x[p,t]
                model += I[p][t] == I[p][t-1] + x[p][t], f"InvBal_{p}_{t}"

        # End of week: inventory + late = demand
        model += I[p][T] + late[p] == demand_week[p], f"Demand_{p}"

    # ----- Machine capacity each day -----
    for m in machines:
        for t in days:
            # Total processing hours on machine m, day t
            proc_hours = pl.lpSum(proc_time[(p, m)] * x[p][t] for p in products)
            # Setup hours = sum over products that are active
            setup_hours = pl.lpSum(y[p][t] for p in products) * setup_time[m]

            model += (
                proc_hours + setup_hours <= hours_per_day[m],
                f"Capacity_{m}_{t}"
            )

    # ----- Link production and setup (big-M) -----
    for p in products:
        for t in days:
            model += x[p][t] <= M_big[p] * y[p][t], f"Link_{p}_{t}"

    # ----- Objective: minimize total cost -----
    total_prod_cost = pl.lpSum(prod_cost[p] * x[p][t] for p in products for t in days)
    total_inv_cost  = pl.lpSum(inv_cost[p]  * I[p][t] for p in products for t in days)
    total_setup_cost = pl.lpSum(setup_cost_value * y[p][t] for p in products for t in days)
    total_late_cost  = pl.lpSum(late_penalty[p] * late[p] for p in products)

    model += total_prod_cost + total_inv_cost + total_setup_cost + total_late_cost

    # ----- Solve -----
    model.solve(pl.PULP_CBC_CMD(msg=False))

    # --------------------------------------------------------
    # 3. BUILD RESULT TABLES
    # --------------------------------------------------------

    # Production schedule
    sched_rows = []
    for p in products:
        for t in days:
            sched_rows.append({
                "scenario": scenario_name,
                "product": p,
                "day": t,
                "qty_produced": x[p][t].value(),
                "inventory_end": I[p][t].value(),
                "setup": y[p][t].value(),
            })
    schedule_df = pd.DataFrame(sched_rows)

    # Machine utilization by day
    util_rows = []
    for m in machines:
        for t in days:
            proc = sum(proc_time[(p, m)] * x[p][t].value() for p in products)
            setups = setup_time[m] * sum(y[p][t].value() for p in products)
            used = proc + setups
            util_rows.append({
                "scenario": scenario_name,
                "machine": m,
                "day": t,
                "proc_hours": proc,
                "setup_hours": setups,
                "used_hours": used,
                "available_hours": hours_per_day[m],
                "utilization": used / hours_per_day[m] if hours_per_day[m] > 0 else 0.0,
            })
    util_df = pd.DataFrame(util_rows)

    # Average utilization per machine over the week
    avg_util = (util_df
                .groupby("machine")
                .agg({"used_hours": "sum", "available_hours": "sum"})
                .reset_index())
    avg_util["scenario"] = scenario_name
    avg_util["avg_utilization"] = avg_util["used_hours"] / avg_util["available_hours"]

    # Late units by product
    late_rows = []
    for p in products:
        late_rows.append({
            "scenario": scenario_name,
            "product": p,
            "late_units": late[p].value(),
        })
    late_df = pd.DataFrame(late_rows)

    # Throughput by product
    thr_rows = []
    for p in products:
        total_produced = sum(x[p][t].value() for t in days)
        thr_rows.append({
            "scenario": scenario_name,
            "product": p,
            "total_produced": total_produced,
        })
    thr_df = pd.DataFrame(thr_rows)

    # KPI summary
    total_cost_val = pl.value(model.objective)
    total_throughput = thr_df["total_produced"].sum()
    total_setups = schedule_df["setup"].sum()
    total_late_units = late_df["late_units"].sum()
    total_demand = sum(demand_week[p] for p in products)
    otd_percent = 100 * (total_demand - total_late_units) / total_demand if total_demand > 0 else 0.0

    kpi = {
        "scenario": scenario_name,
        "total_cost": total_cost_val,
        "total_throughput": total_throughput,
        "total_setups": total_setups,
        "total_late": total_late_units,
        "otd_percent": otd_percent,
    }

    return schedule_df, util_df, avg_util, late_df, thr_df, kpi


# ------------------------------------------------------------
# 3. DEFINE THE THREE SCENARIOS
# ------------------------------------------------------------

results = []

# ---------- SCENARIO A: FLEXIBLE, COMFORTABLE CAPACITY ----------
# Enough hours to make all 3 drinks; setups are cheap.
hours_A = {"M1": 16.0, "M2": 16.0, "M3": 16.0}

# Weekly demand (realistic, fits capacity on M1)
# Total M1 hours needed = 98 out of 112 available.
demand_A = {"P1": 1600, "P2": 1200, "P3": 1000}

# Low setup cost → model is free to be flexible and change products often.
setup_cost_A = 20.0

schedule_A, util_A, avg_A, late_A, thr_A, kpi_A = solve_scenario(
    "A_flexible", hours_A, demand_A, setup_cost_A
)
results.append(kpi_A)

# ---------- SCENARIO B: SETUP-MINIMIZING ----------
# Same capacity and demand as A, but setups are very expensive.
hours_B = {"M1": 16.0, "M2": 16.0, "M3": 16.0}
demand_B = {"P1": 1600, "P2": 1200, "P3": 1000}

# High setup cost pushes batching: fewer changeovers, more inventory.
setup_cost_B = 200.0

schedule_B, util_B, avg_B, late_B, thr_B, kpi_B = solve_scenario(
    "B_setup_heavy", hours_B, demand_B, setup_cost_B
)
results.append(kpi_B)

# ---------- SCENARIO C: TIGHT CAPACITY + RUSH DEMAND ----------
# Less capacity + higher demand → real stress test.
hours_C = {"M1": 12.0, "M2": 12.0, "M3": 12.0}  # cut to 1.5 shifts

# Rush / peak week demand – clearly above comfortable level.
demand_C = {"P1": 2200, "P2": 1600, "P3": 1300}

# Moderate setup cost: we care about both setups and lateness.
setup_cost_C = 80.0

schedule_C, util_C, avg_C, late_C, thr_C, kpi_C = solve_scenario(
    "C_tight_capacity_rush", hours_C, demand_C, setup_cost_C
)
results.append(kpi_C)

kpi_table = pd.DataFrame(results)


# ------------------------------------------------------------
# 4. PRINT RESULTS
# ------------------------------------------------------------

def print_scenario(title, sched, util, avg_util, late, thr, kpi):
    print(f"\n==================== {title} ====================\n")
    print("--- KPI Summary ---")
    print(kpi)

    print("\n--- Production Schedule ---")
    print(sched)

    print("\n--- Machine Utilization ---")
    print(util)

    print("\n--- Average Utilization ---")
    print(avg_util)

    print("\n--- Late Units ---")
    print(late)

    print("\n--- Throughput by Product ---")
    print(thr)


print_scenario("SCENARIO A: FLEXIBLE", schedule_A, util_A, avg_A, late_A, thr_A, kpi_A)
print_scenario("SCENARIO B: SETUP-MINIMIZING", schedule_B, util_B, avg_B, late_B, thr_B, kpi_B)
print_scenario("SCENARIO C: TIGHT CAPACITY + RUSH DEMAND", schedule_C, util_C, avg_C, late_C, thr_C, kpi_C)

print("\n==================== KPI TABLE (ALL SCENARIOS) ====================")
print(kpi_table)



==================== SCENARIO A: FLEXIBLE ====================

--- KPI Summary ---
{'scenario': 'A_flexible', 'total_cost': 5375.00001, 'total_throughput': 3720.0000099999997, 'total_setups': 8.0, 'total_late': 80.0, 'otd_percent': 97.89473684210526}

--- Production Schedule ---
      scenario product  day  qty_produced  inventory_end  setup
0   A_flexible      P1    1     466.66667      466.66667    1.0
1   A_flexible      P1    2     466.66667      933.33333    1.0
2   A_flexible      P1    3     466.66667     1400.00000    1.0
3   A_flexible      P1    4     200.00000     1600.00000    1.0
4   A_flexible      P1    5       0.00000     1600.00000    0.0
5   A_flexible      P1    6       0.00000     1600.00000    0.0
6   A_flexible      P1    7       0.00000     1600.00000    0.0
7   A_flexible      P2    1       0.00000        0.00000    0.0
8   A_flexible      P2    2       0.00000        0.00000    0.0
9   A_flexible      P2    3       0.00000        0.00000    0.0
10  A_flexible